In [1]:
from gensim.models.keyedvectors import KeyedVectors
from tqdm import tqdm

print("считываю кутузовский файл")
path = r"C:\Users\Annie\Downloads\ruwikiruscorpora_upos_skipgram_300_2_2018.vec\ruwikiruscorpora_upos_skipgram_300_2_2018.vec"
w2v = KeyedVectors.load_word2vec_format(path, binary=False)
print("считал")

C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


считываю кутузовский файл
считал


In [ ]:
import pickle
with open('kutuzov.dat', "wb") as file:
    pickle.dump(w2v, file)

In [9]:
import pickle
from gensim.models.keyedvectors import KeyedVectors
from tqdm import tqdm

with open('kutuzov.dat', "rb") as file:
    w2v = pickle.load(file)

In [2]:
import os
import pymorphy2
import re
import pandas as pd
import keras
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

ma = pymorphy2.MorphAnalyzer()

def check_in_kutuzov(word):
    part_of_speech = str(ma.parse(word)[0].tag.POS).upper()
    if part_of_speech == 'ADJF':
        part_of_speech = "ADJ"
    elif part_of_speech == 'ADJS':
        part_of_speech = "ADJ"
    elif part_of_speech == 'NUMR':
        part_of_speech = "NUMR"
    elif part_of_speech == 'ADVB':
        part_of_speech == 'ADV'
    elif part_of_speech == 'INFN' or part_of_speech == 'PRTF' or part_of_speech == 'PRTS':
        part_of_speech = 'VERB'

    new_word = word + "_" + part_of_speech
    if new_word in w2v.vocab:
        return new_word
    else:
        return ""

def clean_text(text):
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text) #deleting symbols 
    text = " ".join(ma.parse(word)[0].normal_form for word in text.split())
    text = ' '.join(word for word in text.split() if len(word)>3)
    all_words = text.split(' ')
    all_correct_words = ""
    for word in all_words:
        new_word = check_in_kutuzov(word)
        if new_word !="":
            if all_correct_words=="":
                all_correct_words += new_word
            else:
                all_correct_words += " "+new_word
    #text = text.encode("utf-8")
    return all_correct_words

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [33]:
#ЗДЕСЬ ПОЛУЧАЕМ ДАТАФРЕЙМ ДЛЯ НАШИХ ТЕКСТОВ
import random

NAME_TRAIN = r'C:\Users\Annie\Documents\Working\Корпус биографических текстов\Данные для обучения сетей\oversampling\ALL.csv'

with open(NAME_TRAIN) as f:
        df = pd.read_csv(f, sep=';', header=None, decimal = ',')
        
train_data = df.values
random.seed(139)  # для воспроизводимость
random.shuffle(train_data)
m = len(train_data)

train_length = int(0.8 * m)
train_data, test_data = train_data[:train_length],\
    train_data[train_length:]

In [23]:
import random
NAME_TRAIN_2=r'C:\Users\Annie\Documents\Working\Корпус биографических текстов\Данные для обучения сетей\oversampling\ALL2.csv'
with open(NAME_TRAIN_2) as f:
        df1 = pd.read_csv(f, sep=';', header=None, decimal = ',')
texts = df1[0].tolist()
categories=df1[1].tolist()        

In [34]:
df3 = pd.DataFrame(test_data)
texts_test = df3[0].tolist()
categories_test = df3[1].tolist()
vectors_test = df3.drop([0,1], axis=1).values

In [35]:
k_parameter = 1
print(M)
sampling_strategy = {0: int(round((M-count[0])*k_parameter+count[0])), 1: int(round((M-count[1])*k_parameter+count[1])), 2: int(round((M-count[2])*k_parameter+count[2])), \
                     3: int(round((M-count[3])*k_parameter+count[3])), 4: int(round((M-count[4])*k_parameter+count[4])), 5: int(round((M-count[5])*k_parameter+count[5])), \
                     6: int(round((M-count[6])*k_parameter+count[6])), 7: int(round((M-count[7])*k_parameter+count[7])), 8: int(round((M-count[8])*k_parameter+count[8])), \
                     9: int(round((M-count[9])*k_parameter+count[9]))}

print(sampling_strategy)

516.0
{0: 516, 1: 516, 2: 516, 3: 516, 4: 516, 5: 516, 6: 516, 7: 516, 8: 516, 9: 516}


In [36]:
#не надо
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words

sw_list = get_stop_words('russian')+get_stop_words('english')

vectorizer = TfidfVectorizer(stop_words=sw_list)
all_words = vectorizer.fit_transform(texts)

C:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [37]:
#не надо
index_value={i[1]:i[0] for i in vectorizer.vocabulary_.items()}

fully_indexed = []
for row in all_words:
    fully_indexed.append({index_value[column]:value for (column,value) in zip(row.indices,row.data)})      

In [38]:
import numpy as np

types_of_facts  = {"birth": 0, "death": 1, "occupation": 2, "education": 3, "affiliation": 4, "family": 5, \
         "parenting": 6, "professional_events": 7, "personal_events": 8, "residence": 9}

In [39]:
#print(texts)
print("training sample:")
num_classes = len(types_of_facts)
count = np.zeros(shape=(num_classes))
for i in range(len(categories)):
    #print(i)
    #print(texts[i])
    texts[i] = clean_text(texts[i])
    categories[i] = types_of_facts[categories[i]]
    count[categories[i]]+=1
print(count)
print("count of sentences: {0}".format(count.sum()))
M=count.max()
print("max class: {0}".format(M))

print("test sample:")
count_test = np.zeros(shape=(num_classes))
for i in range(len(categories_test)):
    texts_test[i] = clean_text(texts_test[i])
    categories_test[i] = types_of_facts[categories_test[i]]
    count_test[categories_test[i]]+=1
print(count_test)

training sample:


KeyError: 4

In [40]:
#не надо
import copy
import math

def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

def get_text(d, word1, word2):
    text=''
    keys=[]
    for v in d.keys():
        keys.append(v)

    for i in range(len(keys)):
        if i==len(d)-1:
            if keys[i] == word1:
                #text+=word2[0]
                text+=word2
            else:
                text+=keys[i]
        else:
            if keys[i] == word1:
                #text+=word2[0]+' '
                text+=word2+' '
            else:
                text+=keys[i]+' '
    return text 

texts_rr = copy.deepcopy(texts)
categories_rr = copy.deepcopy(categories)
print(len(texts))
print(len(texts_rr))

index_max = count.argmax()        
for i in range(len(count)):
    print(i)
    if i != index_max:
        index_curr = count[i]#количество элементов данной категории
        current_array = []
        for j in range(len(fully_indexed)):
            if categories_rr[j] == i:
                current_array.append(fully_indexed[j])
        #print(current_array)
        
        while index_curr < M:
            index = random.randint(0,len(current_array)-1)#номер предложения из выборки
            elem = current_array[index]
            #print("исходное предложение")
            #print(elem)
            
            count_of_words = random.randint(1,math.ceil(len(elem)/2))#сколько слов меняем
            count_of_words=1
            #print("сколько слов меняем")
            #print(count_of_words)
            summa=0
            nn = []
            elem_text=''
            et=0
            for z1 in elem.values():
                poss = (int)(z1*100)
                for z2 in range(poss):
                    nn.append(get_key(elem, z1))
                if et==len(elem.values())-1:
                    elem_text+=get_key(elem, z1)
                else:
                    elem_text+=get_key(elem, z1)+' '
            new_text = elem_text  
            #print(elem_text)
            try:
                for z1 in range(count_of_words):
                    num_word = random.randint(0,len(nn)-1)#рандомлю слово
                    #print("номер слова для замены")
                    #print(num_word)
                    word = nn[num_word]
                    #print("само слово")
                    #print(word)
                    ###получить слово по сходству
                    word1 = word.replace('noun','NOUN')
                    word1 = word1.replace('verb','VERB')
                    word1 = word1.replace('adj','ADJ')
                    count_of_similar = 5
                    num_of_similar = random.randint(0, count_of_similar-1)
                    #print(num_of_similar)
                    #result = w2v.similar_by_word(word1, topn=count_of_similar)
                    #new_word = result[num_of_similar]
                    result = w2v.similar_by_word(word1)
                    #print(result)
                    new_word = result[0][0]
                    #print(new_word)

                    new_text = get_text(elem, word, new_word)
                    #new_text = elem_text.replace(word, new_word[0])
                    new_text = new_text.lower()
                    #print(new_text)
            except:
                print("mist")
                #print(nn)
                #print(elem_text)
                
            texts_rr.append(new_text)
            categories_rr.append(i)
            index_curr+=1  

        
print(len(texts))
print(len(texts_rr))

5160
5160
0
1
2
3
4
5
6
7
8
9
5160
9804


In [41]:
texts

['войти_VERB первоначальный_ADJ состав_NOUN национальный_ADJ комитет_NOUN ссср_NOUN теоретический_ADJ прикладной_ADJ механика_NOUN',
 'войти_VERB первоначальный_ADJ состав_NOUN национальный_ADJ комитет_NOUN ссср_NOUN теоретический_ADJ прикладной_ADJ механика_NOUN',
 'родиться_VERB декабрь_NOUN посёлок_NOUN боржомский_ADJ район_NOUN грузия_NOUN семья_NOUN железнодорожник_NOUN',
 'родиться_VERB декабрь_NOUN посёлок_NOUN боржомский_ADJ район_NOUN грузия_NOUN семья_NOUN железнодорожник_NOUN',
 'войти_VERB первоначальный_ADJ состав_NOUN национальный_ADJ комитет_NOUN ссср_NOUN теоретический_ADJ прикладной_ADJ механика_NOUN',
 'окончить_VERB отделение_NOUN иностранный_ADJ язык_NOUN ленинградский_ADJ государственный_ADJ педагогический_ADJ институт_NOUN герцен_NOUN',
 'родиться_VERB декабрь_NOUN посёлок_NOUN боржомский_ADJ район_NOUN грузия_NOUN семья_NOUN железнодорожник_NOUN',
 'родиться_VERB декабрь_NOUN посёлок_NOUN боржомский_ADJ район_NOUN грузия_NOUN семья_NOUN железнодорожник_NOUN',
 'р

In [42]:
print(count)
index_max = count.argmax()
M = count.max()
N = len(texts)
for i in range(len(count)):
    if i != index_max:
        index_curr = count[i]#количество элементов данной категории
        current_array = []#элементы данной категории
        for j in range(N):#собиараю все элементы категории
            if i == categories[j]:
                current_array.append(texts[j])
        print(len(current_array))
        while index_curr < sampling_strategy[i]:
            index = random.randint(0,len(current_array)-1)
            texts.append(current_array[index])
            categories.append(i)
            index_curr+=1        

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
516
516
516
516
516
516
516
516
516


In [43]:
count1 = np.zeros(shape=(num_classes))
for i in range(len(texts)):
    count1[categories[i]]+=1
print(count1)

[ 516. 1032. 1032. 1032. 1032. 1032. 1032. 1032. 1032. 1032.]


In [44]:
#не надо
random.seed(139) 
random.shuffle(texts_rr)
random.seed(139) 
random.shuffle(categories_rr)
print(texts_rr[100])
print(categories_rr[100])
print(texts[100])

ссср_noun член_noun совет_propn московский_adj медицинский_adj наука_noun общество_noun
4
учиться_VERB послевоенный_ADJ курс_NOUN исторический_ADJ факультет_NOUN обучение_NOUN определить_VERB сфера_NOUN деятельность_NOUN история_NOUN французский_ADJ революция_NOUN


In [45]:
#не надо
for i in range(len(texts_rr)):
    texts_rr[i]=texts_rr[i].replace('_verb','VERB')
    texts_rr[i]=texts_rr[i].replace('_adj','ADJ')
    texts_rr[i]=texts_rr[i].replace('_noun','NOUN')
print(texts_rr[100])

сссрNOUN членNOUN совет_propn московскийADJ медицинскийADJ наукаNOUN обществоNOUN


In [48]:
#for i in range(len(categories)):
    #categories[i] = types_of_facts[categories[i]]
for i in range(len(categories_test)):
    categories_test[i] = types_of_facts[categories_test[i]]

In [49]:
descriptions2 = texts  + texts_test
    
x_train = texts
y_train = categories
    
x_test = texts_test
y_test = categories_test


y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Максимальное количество слов в самом длинном тексте
max_words = 0
for desc in descriptions2:
    words = len(desc.split())
    if words > max_words:
        max_words = words
print('Максимальное количество слов в самом длинном тексте: {} слов'.format(max_words))

maxSequenceLength = max_words

# define class labels
# prepare tokenizer
t = Tokenizer()

for i in range(len(texts)):
    texts[i] = texts[i].replace('_','')
for i in range(len(texts_test)):
    texts_test[i] = texts_test[i].replace('_','')
for i in range(len(descriptions2)):
    descriptions2[i] = descriptions2[i].replace('_','')
    
t.fit_on_texts(descriptions2)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs_train = t.texts_to_sequences(x_train)
encoded_docs_test = t.texts_to_sequences(x_test)
# pad documents to a max length of 4 words

padded_docs_train = sequence.pad_sequences(encoded_docs_train, maxlen=maxSequenceLength)
padded_docs_test = sequence.pad_sequences(encoded_docs_test, maxlen=maxSequenceLength)

total_unique_words = len(t.word_counts)
print('Всего уникальных слов в словаре: {}'.format(total_unique_words))

Максимальное количество слов в самом длинном тексте: 71 слов
Всего уникальных слов в словаре: 4844


In [50]:
import numpy as np
#ДОЛГОЕ ПОСТРОЕНИЕ МАТРИЦЫ ВЕСОВ
# create a weight matrix for words in training docs
print("Создаю матрицу")
embedding_matrix = np.zeros((vocab_size, 300))
print("Начинаем")

for word, i in t.word_index.items():
    print(word)
    if i%100 == 0:
        print("checking {0}/{1}".format(i, len(t.word_index.items())))
    '''
    new_word = word.replace('adj', '_ADJ')
    new_word = new_word.replace('noun', '_NOUN')
    new_word = new_word.replace('verb', '_VERB')
    embedding_vector = w2v[new_word]

    if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    '''
    try:
        new_word = word.replace('adj', '_ADJ')
        new_word = new_word.replace('noun', '_NOUN')
        new_word = new_word.replace('verb', '_VERB')
        embedding_vector = w2v[new_word]

        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        
    except:
        print(word)
        print(i) 
    

Создаю матрицу
Начинаем
университетnoun
членnoun
в
в
3
родитьсяverb
окончитьverb
адамсnoun
сентябрьnoun
работатьverb
ребёнокnoun
июньnoun
кейптаунскийadj
семьяnoun
институтnoun
новыйadj
званиеnoun
михаилnoun
войнаnoun
московскийadj
инженерnoun
зеландияnoun
исполкомnoun
академияnoun
директорnoun
местоnoun
получитьverb
диссертацияnoun
йоркnoun
москваnoun
областьnoun
научныйadj
иосифnoun
умеретьverb
профессорnoun
деятельностьnoun
женаnoun
изгнаниеnoun
статьverb
заменитьverb
советnoun
сссрnoun
колонизацияnoun
другnoun
среднийadj
историяnoun
выставкаnoun
монастырьnoun
житьverb
школаnoun
старшийnoun
августnoun
иоганнnoun
обществоnoun
красныйadj
испанияnoun
монахnoun
дочьnoun
парижnoun
сестраnoun
государственныйadj
докторскийadj
городnoun
докторnoun
женитьсяverb
январьnoun
представитьverb
отправитьverb
факультетnoun
диссидентnoun
дианаnoun
сингапурскийadj
ноябрьnoun
октябрьnoun
группаnoun
и
и
74
российскийadj
направитьverb
наукаnoun
великобританияnoun
русскийadj
советскийadj
жительnoun
джеймс

физическийadj
медицинаnoun
м
м
628
филологическийadj
обсерваторияnoun
продолжитьverb
сельскийadj
лётчикnoun
владиславnoun
планерныйadj
преподавательnoun
лондонnoun
нидерландыnoun
начатьсяverb
актёрnoun
института
института
641
к
к
642
заочныйadj
маркnoun
пушкиноnoun
медальnoun
возглавлятьverb
финансовыйadj
центрnoun
управлениеnoun
за
за
651
после
после
652
преподавательскийadj
подготовкаnoun
качествоnoun
магистрnoun
мастерскаяnoun
семинарияnoun
борисnoun
анализnoun
агапnoun
призватьverb
ломоносовnoun
боевойadj
член
член
665
крейнnoun
революцияnoun
духовныйadj
лётныйadj
особенностьnoun
возрастnoun
для
для
672
механикоnoun
отправитьсяverb
кавказскийadj
горькийnoun
конкурсnoun
партийныйadj
гагаринnoun
уссрnoun
художественныйadj
гражданскийadj
послатьverb
комиссияnoun
дипломnoun
защитаnoun
отрядnoun
неизвестныйadj
дипломироватьverb
раввинnoun
придерживатьсяverb
предписаниеnoun
иудаизмnoun
датьverb
методnoun
выборnoun
юрийnoun
философияnoun
инструкторnoun
оксфордскийadj
checking 700/4844
рук

эффективностьnoun
кибернетикnoun
ростовскийadj
сибирьnoun
сельвинскийadj
checking 1200/4844
тульскийadj
бюроnoun
литературныйadj
технологияnoun
похоронитьverb
надгробныйadj
плитаnoun
захоронениеnoun
григорийnoun
миссnoun
эмблемаnoun
прогрессивныйadj
символnoun
сингапурnoun
трубаnoun
суджанскийadj
выпускникnoun
ивановоnoun
чечняnoun
лактонnoun
анриnoun
естествознаниеnoun
женеваnoun
детствоnoun
предложитьverb
помехаnoun
аспирантnoun
аугсбургскийadj
миннеаполисnoun
документnoun
босойadj
верныйadj
техасnoun
энергетическийadj
редакцияnoun
осеньnoun
кембриджскийadj
ейскийadj
комарnoun
дерптскийadj
раститьverb
валентинаnoun
находитьverb
совершенствоватьсяverb
рождественскийnoun
профессорскийadj
стипендиатnoun
патрологияnoun
морфологическийadj
характеристикаnoun
лечениеnoun
периферическийadj
конструкторnoun
предприятиеnoun
автозаводnoun
фордnoun
детройтnoun
инструментальныйadj
домашнийadj
госпитальnoun
администрацияnoun
отказыватьverb
конецnoun
иркутnoun
мнитьverb
губкинnoun
периодnoun
востокn

сочинениеnoun
округnoun
луисnoun
позволитьverb
захватитьverb
вражескийadj
рубежnoun
режиссёрскийadj
пролеткультnoun
дворецnoun
лейтенантnoun
инициативаnoun
возвращениеnoun
поверкаnoun
границаnoun
возобновлениеnoun
пограничныйadj
знакnoun
норвегияnoun
ивановичnoun
юзефnoun
объявитьverb
телохранительnoun
открытьverb
пригласитьverb
путинnoun
избиратьсяverb
ответственныйadj
информационныйadj
агентствоnoun
телекомпанияnoun
кружокnoun
официальныйadj
пожалованиеnoun
дворянствоnoun
кампанецnoun
елизаветаnoun
петровичnoun
сольныйadj
концертnoun
репертуарnoun
продвинутьverb
триестnoun
эмигрантnoun
европаnoun
операцияnoun
ярославскийadj
зоологnoun
геоботаническийadj
перевестиverb
моторостроениеnoun
действоватьverb
вылетnoun
нерчинскийadj
тункинскийadj
собратьverb
значительныйadj
коллекцияnoun
планироватьverb
становойnoun
хребетnoun
острогnoun
осуществитьсяverb
ледовитыйadj
океанnoun
результатnoun
отысканиеnoun
тунгусnoun
устьеnoun
ленаnoun
целоеnoun
скелетnoun
шерстистыйadj
частьnoun
checking 180

гидравликаnoun
сибирскийadj
лесоводствоnoun
омскnoun
победаnoun
летательныйadj
аппаратnoun
министрnoun
древнийadj
марксистскийadj
демократnoun
оттоnoun
бауэрnoun
рудольфnoun
оформитьverb
демократическийadj
земляnoun
перевязочныйadj
шовныйadj
полимерныйadj
загсnoun
погрестиverb
проводитьverb
упорядочиватьverb
монашескийadj
долгийadj
песняnoun
альбомnoun
всемирныйadj
экономистnoun
рольnoun
сыгратьverb
угрожатьverb
привлечениеnoun
тунеядствоnoun
испанскийadj
хирургnoun
любимецnoun
севильскийadj
бандитnoun
говоритьverb
куйбышевnoun
начинатьverb
летоnoun
считатьverb
предыдущийadj
вьетнамnoun
сталинградскийadj
литераторскийadj
питерnoun
аграnoun
нобелевскийadj
разделитьverb
американецnoun
командироватьverb
приездnoun
сетчатокрылыйadj
перепончатокрылыйadj
отношениеnoun
аграрныйadj
госдумаnoun
противодействиеnoun
коррупцияnoun
религияnoun
загребnoun
прозекторnoun
больницаnoun
патологическийadj
анатомияnoun
подписатьverb
декретnoun
комиссариатnoun
пополнитьсяverb
радостныйadj
песенкаnoun
сделат

изданиям
изданиям
2620
«вестник
«вестник
2621
еврейской
еврейской
2622
советской
советской
2623
культуры»
культуры»
2624
др
др
2625
1957
1957
2626
«партии
«партии
2627
власти»
власти»
2628
формулировкой
формулировкой
2629
собственному
собственному
2630
1990
1990
2631
кавалерийской
кавалерийской
2632
литературная
литературная
2633
адариди
адариди
2634
1880
1880
2635
статьей
статьей
2636
пор
пор
2637
непрерывно
непрерывно
2638
продолжалась
продолжалась
2639
«военном
«военном
2640
сборнике»
сборнике»
2641
«разведчике»
«разведчике»
2642
учителем
учителем
2643
1922
1922
2644
«високосное
«високосное
2645
лето»
лето»
2646
ефремовым
ефремовым
2647
уходит
уходит
2648
один
один
2649
высшее
высшее
2650
образование
образование
2651
государственную
государственную
2652
сооружений
сооружений
2653
систем
систем
2654
1976
1976
2655
1969
1969
2656
экспедиции
экспедиции
2657
николаю
николаю
2658
академик
академик
2659
радиостанции
радиостанции
2660
начал
начал
2661
руководитель
руководитель
2662
известн

теологии
3088
собственной
собственной
3089
книге
книге
3090
сведений
сведений
3091
астрономии
астрономии
3092
обнаруживается
обнаруживается
3093
наставником
наставником
3094
адамсонa
адамсонa
3095
лрофессор
лрофессор
3096
джон
джон
3097
сезнек
сезнек
3098
адамсон
адамсон
3099
бальзак
checking 3100/4844
бальзак
3100
изобразительное
изобразительное
3101
искусство
искусство
3102
докучаева
докучаева
3103
павлович
павлович
3104
редактора
редактора
3105
сборника
сборника
3106
«материалы
«материалы
3107
изучению
изучению
3108
почв»
почв»
3109
советовым
советовым
3110
выпуски
выпуски
3111
земятченским
земятченским
3112
людвиг
людвиг
3113
адамович
адамович
3114
1823
1823
3115
имеет
имеет
3116
рождения
рождения
3117
питер
питер
3118
агре
агре
3119
городке
городке
3120
нортфилд
нортфилд
3121
штате
штате
3122
миннесота
миннесота
3123
борис
борис
3124
кузык
кузык
3125
1958
1958
3126
посёлке
посёлке
3127
александровка
александровка
3128
кировоградской
кировоградской
3129
428
428
3130
борьбе
борьбе
3

странах
странах
3429
снг
снг
3430
крупнейшую
крупнейшую
3431
отечественную
отечественную
3432
патофизиологов
патофизиологов
3433
кончились
кончились
3434
патроны
патроны
3435
курбатов
курбатов
3436
подорвал
подорвал
3437
машину
машину
3438
минами
минами
3439
уничтожив
уничтожив
3440
десятков
десятков
3441
солдат
солдат
3442
противника
противника
3443
погибнув
погибнув
3444
нуха
нуха
3445
шеки
шеки
3446
чиновника
чиновника
3447
гасана
гасана
3448
джаффар
джаффар
3449
оглы
оглы
3450
абдурахманова
абдурахманова
3451
литмо
литмо
3452
1954
1954
3453
академика
академика
3454
крылова
крылова
3455
молекулярной
молекулярной
3456
биологии
биологии
3457
новодевичьем
новодевичьем
3458
оао
оао
3459
«междуреченская
«междуреченская
3460
угольная
угольная
3461
компания
компания
3462
96»
96»
3463
заграничной
заграничной
3464
францию
францию
3465
альбером
альбером
3466
собулем
собулем
3467
ввел
ввел
3468
круг
круг
3469
историков
историков
3470
«якобинского»
«якобинского»
3471
направления
направления
347

синтаксической
синтаксической
3786
класс»
класс»
3787
гильфердингом
гильфердингом
3788
учреждает
учреждает
3789
«магх
«магх
3790
studien»
studien»
3791
редактором
редактором
3792
которого
которого
3793
1904
1904
3794
вышла
вышла
3795
замуж
замуж
3796
философа
философа
3797
роберта
роберта
3798
меррихью
меррихью
3799
малевичем
checking 3800/4844
малевичем
3800
суетиным
суетиным
3801
харджиевым
харджиевым
3802
эренбургом
эренбургом
3803
поддерживал
поддерживал
3804
ними
ними
3805
постоянную
постоянную
3806
переписку
переписку
3807
трений
трений
3808
ситковецким
ситковецким
3809
крисом
крисом
3810
кельми
кельми
3811
оказывается
оказывается
3812
грани
грани
3813
распада
распада
3814
барабанщиком
барабанщиком
3815
«високосного
«високосного
3816
лета»
лета»
3817
валерием
валерием
3818
макаревичу
макаревичу
3819
остался
остался
3820
ссоры
ссоры
3821
последовавшего
последовавшего
3822
развала
развала
3823
техническое
техническое
3824
приёмке
приёмке
3825
эксплуатацию
эксплуатацию
3826
инженерн

4144
«ядра
«ядра
4145
частицы
частицы
4146
поля»
поля»
4147
агбунов
агбунов
4148
погиб
погиб
4149
невыясненных
невыясненных
4150
обстоятельствах
обстоятельствах
4151

мни
4290
ргу
ргу
4291
нефти
нефти
4292
газа
газа
4293
губкина
губкина
4294
французского
французского
4295
товарищем
товарищем
4296
герольдмейстера
герольдмейстера
4297
герольдмейстером
герольдмейстером
4298
1753—1755
1753—1755
4299
b
checking 4300/4844
b
4300
марте
марте
4301
отобран
отобран
4302
отряд
отряд
4303
космонавтов
космонавтов
4304
июлe
июлe
4305
кандидата
кандидата
4306
космонавты
космонавты
4307
испытатели
испытатели
4308
291
291
4309
нпо
нпо
4310
«энергия»
«энергия»
4311
вернувшись
вернувшись
4312
родину
родину
4313
бухгалтером
бухгалтером
4314
сберкассой
сберкассой
4315
парторгом
парторгом
4316
совхоза
совхоза
4317
лесничим
лесничим
4318
трёхмесячные
трёхмесячные
4319
санитаров
санитаров
4320
инструкторов
инструкторов
4321
выполнил
выполнил
4322
днепрогэс
днепрогэс
4323
братской
братской
4324
ингурской
ингур

In [51]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Flatten, GRU, SimpleRNN
from keras.optimizers import RMSprop
from keras.layers import Bidirectional

print(maxSequenceLength)
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False))
#model.add(e)
#e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)
#model.add(e)
#model.add(Flatten())
#model.add(Dense(200, activation='sigmoid'))
#model.add(Dropout=0.5)
#model.add(Embedding(300, maxSequenceLength))
model.add(Bidirectional(LSTM(128, dropout=0.5, recurrent_dropout=0.2)))
#model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.2))

model.add(Dense(num_classes, activation='softmax'))
# compile the model
#rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-6)
#model.compile(optimizer = rmsprop, loss = 'mean_squared_error', metrics=['mean_squared_error', 'mae'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 71, 300)           1453500   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total params: 1,895,366
Trainable params: 441,866
Non-trainable params: 1,453,500
_________________________________________________________________
None


In [52]:
# fit the model
history = model.fit(padded_docs_train, y_train, epochs = 20, verbose=2, validation_data=(padded_docs_test, y_test))

import matplotlib.pyplot as plt

Train on 9804 samples, validate on 395 samples
Epoch 1/20
 - 190s - loss: 1.0981 - acc: 0.6627 - val_loss: 3.0196 - val_acc: 0.0152
Epoch 2/20
 - 177s - loss: 0.4740 - acc: 0.8563 - val_loss: 2.7245 - val_acc: 0.0152
Epoch 3/20
 - 175s - loss: 0.3739 - acc: 0.8873 - val_loss: 2.4986 - val_acc: 0.0152
Epoch 4/20
 - 176s - loss: 0.3125 - acc: 0.9036 - val_loss: 2.4748 - val_acc: 0.0152
Epoch 5/20
 - 175s - loss: 0.2866 - acc: 0.9145 - val_loss: 2.4112 - val_acc: 0.0152
Epoch 6/20
 - 177s - loss: 0.2631 - acc: 0.9214 - val_loss: 2.4023 - val_acc: 0.0152
Epoch 7/20
 - 175s - loss: 0.2365 - acc: 0.9264 - val_loss: 2.3720 - val_acc: 0.0152
Epoch 8/20
 - 175s - loss: 0.2110 - acc: 0.9350 - val_loss: 2.3799 - val_acc: 0.0152
Epoch 9/20
 - 175s - loss: 0.1937 - acc: 0.9403 - val_loss: 2.3262 - val_acc: 0.0152
Epoch 10/20
 - 178s - loss: 0.1830 - acc: 0.9430 - val_loss: 2.2929 - val_acc: 0.0152
Epoch 11/20
 - 184s - loss: 0.1686 - acc: 0.9473 - val_loss: 2.2569 - val_acc: 0.0152
Epoch 12/20
 - 1

In [53]:
# evaluate the model

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

for  i in range(20):
    history = model.fit(padded_docs_train, y_train, epochs = 1, verbose=2, validation_data=(padded_docs_test, y_test))



    predict = np.argmax(model.predict(padded_docs_test), axis=1)
    answer = np.argmax(y_test, axis=1)

    print('Accuracy: %f' % (accuracy_score(predict, answer)*100))
    print('F1-score: %f' % (f1_score(predict, answer, average="macro")*100))
    #print('Precision: %f' % (precision_score(predict, answer, average="macro")*100))
    #print('Recall: %f' % (recall_score(predict, answer, average="macro")*100))

Train on 9804 samples, validate on 395 samples
Epoch 1/1
 - 172s - loss: 0.0748 - acc: 0.9793 - val_loss: 2.4765 - val_acc: 0.0152
Accuracy: 1.518987


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


F1-score: 0.299252
Train on 9804 samples, validate on 395 samples
Epoch 1/1
 - 172s - loss: 0.0688 - acc: 0.9798 - val_loss: 2.5710 - val_acc: 0.0152
Accuracy: 1.518987
F1-score: 0.299252
Train on 9804 samples, validate on 395 samples
Epoch 1/1
 - 173s - loss: 0.0588 - acc: 0.9830 - val_loss: 2.4916 - val_acc: 0.0152
Accuracy: 1.518987
F1-score: 0.299252
Train on 9804 samples, validate on 395 samples
Epoch 1/1
 - 172s - loss: 0.0597 - acc: 0.9813 - val_loss: 2.5482 - val_acc: 0.0152
Accuracy: 1.518987
F1-score: 0.299252
Train on 9804 samples, validate on 395 samples
Epoch 1/1
 - 172s - loss: 0.0550 - acc: 0.9824 - val_loss: 2.5373 - val_acc: 0.0152
Accuracy: 1.518987
F1-score: 0.299252
Train on 9804 samples, validate on 395 samples
Epoch 1/1
 - 172s - loss: 0.0457 - acc: 0.9861 - val_loss: 2.5926 - val_acc: 0.0152
Accuracy: 1.518987
F1-score: 0.299252
Train on 9804 samples, validate on 395 samples
Epoch 1/1
 - 176s - loss: 0.0440 - acc: 0.9855 - val_loss: 2.5838 - val_acc: 0.0152
Accur